In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics       import mean_absolute_error,r2_score
from matplotlib import pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', None)

look_back=5000 # ~Past 6 months data to train on
look_ahead=24 # Predict next look_ahead values

2022-10-11 23:07:45.779850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-11 23:07:46.833226: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-11 23:07:46.833261: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-11 23:07:46.966667: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-11 23:07:49.139045: W tensorflow/stream_executor/platform/de

In [ ]:
# Loading the data
data = pd.read_csv('data.csv',index_col=0)

# [Feature Engineering]
data.loc[data['ont_ene']<0,'ont_ene']=0

mean=data['ont_ene'].mean() # Clipping outliers using zscore analysis
std=data['ont_ene'].std()

upperlimit=mean + 3 * std
data.loc[data['ont_ene']>upperlimit,'ont_ene']=upperlimit # Can try setting it None and fill forward

In [ ]:
# [Train Test Split]
X=data.dropna().iloc[-look_back:,:-1].values # Keeping all the non-null rows as training data
y=data.dropna().iloc[-look_back:,-1].values
test=data[data.ont_ene.isnull()].iloc[:look_ahead,:-1].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train=np.reshape(y_train, (-1,1))
y_val=np.reshape(y_val, (-1,1))

In [ ]:
sc_x = MinMaxScaler(feature_range = (0, 1))
X_train = sc_x.fit_transform(X_train)
X_val = sc_x.transform(X_val)

sc_y = MinMaxScaler(feature_range = (0, 1))
y_train = sc_y.fit_transform(y_train)
y_val = sc_y.transform(y_val)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

model.compile(loss = 'mae', optimizer = 'adam', metrics = ['mae'])

2022-10-11 23:08:12.583944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 23:08:12.584582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-11 23:08:12.584784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-11 23:08:12.584937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-11 23:08:12.585056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [ ]:
history=model.fit(X_train, y_train, batch_size = 32, epochs = 200,validation_data=(X_val, y_val))

Epoch 1/200
609/609 [==============================] - 2s 3ms/step - loss: 0.0753 - mae: 0.0753 - val_loss: 0.0655 - val_mae: 0.0655
Epoch 2/200
609/609 [==============================] - 2s 3ms/step - loss: 0.0636 - mae: 0.0636 - val_loss: 0.0593 - val_mae: 0.0593
Epoch 3/200
609/609 [==============================] - 2s 3ms/step - loss: 0.0609 - mae: 0.0609 - val_loss: 0.0605 - val_mae: 0.0605
Epoch 4/200
609/609 [==============================] - 2s 3ms/step - loss: 0.0594 - mae: 0.0594 - val_loss: 0.0569 - val_mae: 0.0569
Epoch 5/200
609/609 [==============================] - 2s 2ms/step - loss: 0.0583 - mae: 0.0583 - val_loss: 0.0592 - val_mae: 0.0592
Epoch 6/200
609/609 [==============================] - 1s 2ms/step - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0562 - val_mae: 0.0562
Epoch 7/200
609/609 [==============================] - 2s 3ms/step - loss: 0.0570 - mae: 0.0570 - val_loss: 0.0558 - val_mae: 0.0558
Epoch 8/200
609/609 [==============================] - 2s 3ms/step - 

In [ ]:
X_test = sc_x.transform(test)
y_pred=model.predict(X_test)

1/1 [==============================] - 0s 99ms/step


In [ ]:
y_preds=sc_y.inverse_transform(y_pred)
y_true=sc_y.inverse_transform(y_val)

In [ ]:
pd.DataFrame(list(zip(y_true[:24],y_preds[:24])), columns = ['Actual Values', 'Predicted Values'])

,Actual Values,Predicted Values
0,[59.17],[47.98119]
1,[101.75],[46.725418]
2,[10.31],[42.23819]
3,[13.21],[36.608658]
4,[12.27],[56.218853]
5,[13.47],[70.16918]
6,[0.0],[70.65171]
7,[37.45],[59.71214]
8,[39.22],[37.169163]
9,[0.0],[36.112976]


In [ ]:
mean_absolute_error(y_true[:24],y_preds[:24])

124.26970881859461

In [ ]:
r2_score(y_true[:24],y_preds[:24])

-382.63229075455087

In [18]:
pd.DataFrame(list(zip(y_true[:24],y_preds[:24])), columns = ['Actual Values', 'Predicted Values'])

,Actual Values,Predicted Values
0,[70.97],[10.131966]
1,[53.88],[2.9973075]
2,[32.15],[11.653866]
3,[12.11],[49.043747]
4,[0.0],[50.751644]
5,[43.17],[55.553806]
6,[12.81],[60.86021]
7,[60.46],[52.29269]
8,[50.9],[52.588547]
9,[11.68],[50.801884]


In [ ]:
y_true=np.array([14.35,14.36,11.64,7.09,41.7,61.4,56.92]).reshape(-1,1)

In [ ]:
y_true

array([[14.35],
       [14.36],
       [11.64],
       [ 7.09],
       [41.7 ],
       [61.4 ],
       [56.92]])

In [ ]:
mean_absolute_error(y_true[:7],y_preds[:7])

23.304742954799103

In [24]:
std

32.867150967817736